In [41]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier 

import subprocess
import os 

In [47]:
preprop_test = False
preprop_suppl = False 

In [48]:
folder = os.path.join( os.getcwd(), 'test' ) 
test_out = os.path.join( os.getcwd(), 'test.csv')
rscript_file= os.path.join( os.getcwd(), 'features.R')

if preprop_test:
    subprocess.call(["Rscript", rscript_file, folder,test_out],stdin=subprocess.PIPE, stdout=subprocess.PIPE,stderr=subprocess.STDOUT)
    

In [37]:
train = pd.read_csv("train.csv")
test = pd.read_csv(test_out,index_col='sound.files')

print(train.shape,test.shape)
test.head(2)

(277, 21) (125, 20)


,meanfreq,sd,median,Q25,Q75,IQR,skew,kurt,sp.ent,sfm,mode,centroid,meanfun,minfun,maxfun,meandom,mindom,maxdom,dfrange,modindx
sound.files,,,,,,,,,,,,,,,,,,,,
CEO_F_XRX.wav,0.188060,0.060616,0.192284,0.168280,0.22344,0.055160,3.195981,21.594193,0.931335,0.527679,0.001000,0.188060,0.142097,0.047337,0.279070,0.674508,0.0,6.304688,6.304688,0.071972
CEO_M_ACM_20121204.wav,0.158859,0.064130,0.136024,0.116171,0.22224,0.106069,2.603506,12.043091,0.940926,0.547693,0.119521,0.158859,0.113010,0.043235,0.279114,0.323591,0.0,5.038770,5.038770,0.052861


In [38]:
train.loc[train['label']=="male",'label'] = 0
train.loc[train['label']=="female",'label'] = 1

scaler = StandardScaler()
scaler.fit(train.iloc[:,0:20])
X_train = scaler.transform(train.iloc[:,0:20])
X_test = scaler.transform(test.iloc[:,0:20])
y_train = list(train['label'].values)

In [39]:
# LogisticRegression
classifiers = {}

classifiers['lr'] = LogisticRegression().fit(X_train, y_train)
classifiers['dtree'] = DecisionTreeClassifier().fit(X_train, y_train)
classifiers['rf'] = RandomForestClassifier().fit(X_train, y_train)
classifiers['xgb'] = XGBClassifier().fit(X_train, y_train)

for c in classifiers:
    test[c+'_mprob'] = [ i[0] for i in classifiers[c].predict_proba(X_test) ] 


In [40]:
test.head(2)

,meanfreq,sd,median,Q25,Q75,IQR,skew,kurt,sp.ent,sfm,...,maxfun,meandom,mindom,maxdom,dfrange,modindx,lr_mprob,dtree_mprob,rf_mprob,xgb_mprob
sound.files,,,,,,,,,,,,,,,,,,,,,
CEO_F_XRX.wav,0.188060,0.060616,0.192284,0.168280,0.22344,0.055160,3.195981,21.594193,0.931335,0.527679,...,0.279070,0.674508,0.0,6.304688,6.304688,0.071972,0.083875,0.0,0.02,0.001204
CEO_M_ACM_20121204.wav,0.158859,0.064130,0.136024,0.116171,0.22224,0.106069,2.603506,12.043091,0.940926,0.547693,...,0.279114,0.323591,0.0,5.038770,5.038770,0.052861,0.836356,1.0,0.87,0.940854
